<a href="https://colab.research.google.com/github/mishu11/Python/blob/main/spark_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#pySpark

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 199 kB 30.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=b9f86f2af7ddbde8a912fb267ee83f8a73b1b05bdc673ad95a43470342695fc1
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
import numpy as np


In [ ]:
spark = SparkSession.builder.appName('Spark Practice').master('local[*]').getOrCreate()

In [ ]:
a = to_date('2021-04-30')
dates = spark.sql(f"""select last_day(to_date({a})) as start_date, last_day(to_date('2021-04-30')-interval 1 month) as end_date;""")
sd = dates.toPandas()['start_date'][0]
ed = dates.toPandas()['end_date'][0]
print(sd, ed)

ParseException: ignored

In [ ]:
a = spark.sql(f"""select current_date()>to_date({sd}) as date1""")
a.show()

+-----+
|date1|
+-----+
| true|
+-----+



In [ ]:
arrayData = [
    (("Saif", "", "Shaikh"), ["English", "Science", "Maths"], "HYD", "M", 1000),
    (("Ram", "Sachin", ""), ["Spark", "English", "Maths"], "BLR", "F", 10000),
    (("Aniket", "", "Mishra"), ["Civics", "History"], "HYD", "F", 2000),
    (("Mitali", "Sahil", "Kashiv"), ["Civics", "History"], "BLR", "M", 5000),
    (("Zaid", "Riyaz", "Shaikh"), ["Civics", "History"], "BLR", "M", 4000),
    (("Sufi", "Alim", "Shaikh"), ["Hindi", "History"], "HYD", "M", 1000),
    (("Saif", "", "Shaikh"), ["English", "Science", "Maths"], "HYD", "M", 1000)
    ]

sch = StructType([
    StructField('Full_Name', StructType([
        StructField('fname', StringType(), True), 
        StructField('mname', StringType(), True), 
        StructField('lname', StringType(), True)]),True),
    StructField('Subject', ArrayType(StringType()), True),
    StructField('State', StringType(), True),
    StructField('Gender', StringType(), True),
    StructField('Sal', IntegerType(), True)
])

df = spark.createDataFrame(data=arrayData, schema=sch)

In [ ]:
df.show()
df.printSchema()

+--------------------+--------------------+-----+------+-----+
|           Full_Name|             Subject|State|Gender|  Sal|
+--------------------+--------------------+-----+------+-----+
|    {Saif, , Shaikh}|[English, Science...|  HYD|     M| 1000|
|     {Ram, Sachin, }|[Spark, English, ...|  BLR|     F|10000|
|  {Aniket, , Mishra}|   [Civics, History]|  HYD|     F| 2000|
|{Mitali, Sahil, K...|   [Civics, History]|  BLR|     M| 5000|
|{Zaid, Riyaz, Sha...|   [Civics, History]|  BLR|     M| 4000|
|{Sufi, Alim, Shaikh}|    [Hindi, History]|  HYD|     M| 1000|
|    {Saif, , Shaikh}|[English, Science...|  HYD|     M| 1000|
+--------------------+--------------------+-----+------+-----+

root
 |-- Full_Name: struct (nullable = true)
 |    |-- fname: string (nullable = true)
 |    |-- mname: string (nullable = true)
 |    |-- lname: string (nullable = true)
 |-- Subject: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- State: string (nullable = true)
 |-- Gender

In [ ]:
df.withColumn('Gender', when(col('Gender')=='M', 'Male')\
                    .when(col('Gender')=='F', 'Female')\
                    .otherwise('Unknown')
                    ).show()

+--------------------+--------------------+-----+------+-----+
|           Full_Name|             Subject|State|Gender|  Sal|
+--------------------+--------------------+-----+------+-----+
|    {Saif, , Shaikh}|[English, Science...|  HYD|  Male| 1000|
|     {Ram, Sachin, }|[Spark, English, ...|  BLR|Female|10000|
|  {Aniket, , Mishra}|   [Civics, History]|  HYD|Female| 2000|
|{Mitali, Sahil, K...|   [Civics, History]|  BLR|  Male| 5000|
|{Zaid, Riyaz, Sha...|   [Civics, History]|  BLR|  Male| 4000|
|{Sufi, Alim, Shaikh}|    [Hindi, History]|  HYD|  Male| 1000|
|    {Saif, , Shaikh}|[English, Science...|  HYD|  Male| 1000|
+--------------------+--------------------+-----+------+-----+



In [ ]:
n_df = df.select(df.Full_Name.fname.alias('FName'), \
          df.Full_Name.mname.alias('MName'), \
          df.Full_Name.lname.alias('LName'), \
          df.State.alias('State'), \
          df.Gender.alias('Gender'), \
          df.Sal.alias('Salary'), \
          explode('Subject').alias('Subject')
          )

In [ ]:
n_df.sort('FName').show()

+------+------+------+-----+------+------+-------+
| FName| MName| LName|State|Gender|Salary|Subject|
+------+------+------+-----+------+------+-------+
|Aniket|      |Mishra|  HYD|     F|  2000| Civics|
|Aniket|      |Mishra|  HYD|     F|  2000|History|
|Mitali| Sahil|Kashiv|  BLR|     M|  5000| Civics|
|Mitali| Sahil|Kashiv|  BLR|     M|  5000|History|
|   Ram|Sachin|      |  BLR|     F| 10000|  Spark|
|   Ram|Sachin|      |  BLR|     F| 10000|English|
|   Ram|Sachin|      |  BLR|     F| 10000|  Maths|
|  Saif|      |Shaikh|  HYD|     M|  1000|  Maths|
|  Saif|      |Shaikh|  HYD|     M|  1000|Science|
|  Saif|      |Shaikh|  HYD|     M|  1000|English|
|  Saif|      |Shaikh|  HYD|     M|  1000|Science|
|  Saif|      |Shaikh|  HYD|     M|  1000|English|
|  Saif|      |Shaikh|  HYD|     M|  1000|  Maths|
|  Sufi|  Alim|Shaikh|  HYD|     M|  1000|History|
|  Sufi|  Alim|Shaikh|  HYD|     M|  1000|  Hindi|
|  Zaid| Riyaz|Shaikh|  BLR|     M|  4000| Civics|
|  Zaid| Riyaz|Shaikh|  BLR|   

To Pandas Df

In [ ]:
pdDf = n_df.toPandas().replace('', 'A')
pdDf

,FName,MName,LName,State,Gender,Salary,Subject
0,Saif,A,Shaikh,HYD,M,1000,English
1,Saif,A,Shaikh,HYD,M,1000,Science
2,Saif,A,Shaikh,HYD,M,1000,Maths
3,Ram,Sachin,A,BLR,F,10000,Spark
4,Ram,Sachin,A,BLR,F,10000,English
5,Ram,Sachin,A,BLR,F,10000,Maths
6,Aniket,A,Mishra,HYD,F,2000,Civics
7,Aniket,A,Mishra,HYD,F,2000,History
8,Mitali,Sahil,Kashiv,BLR,M,5000,Civics
9,Mitali,Sahil,Kashiv,BLR,M,5000,History


In [ ]:
new_df = n_df.withColumn('MName', when(col('MName')=='', 'A').otherwise(col('MName')))

In [ ]:
new_df.select('MName', length('MName').alias('Len_MName')).distinct()\
.sort('Len_MName', ascending=False).show()

+------+---------+
| MName|Len_MName|
+------+---------+
|Sachin|        6|
| Sahil|        5|
| Riyaz|        5|
|  Alim|        4|
|     A|        1|
+------+---------+



In [ ]:
new_df.groupBy('MName').sum().show()

+------+-----------+
| MName|sum(Salary)|
+------+-----------+
|     A|      10000|
|Sachin|      30000|
| Sahil|      10000|
| Riyaz|       8000|
|  Alim|       2000|
+------+-----------+



In [ ]:
import numpy as np
new_df.withColumn('MName', when(col('MName')=='', np.NaN)\
              .otherwise(col('MName'))\
              .alias('New_MName')
).show()

+------+------+------+-----+------+------+-------+
| FName| MName| LName|State|Gender|Salary|Subject|
+------+------+------+-----+------+------+-------+
|  Saif|   NaN|Shaikh|  HYD|     M|  1000|English|
|  Saif|   NaN|Shaikh|  HYD|     M|  1000|Science|
|  Saif|   NaN|Shaikh|  HYD|     M|  1000|  Maths|
|   Ram|Sachin|      |  BLR|     F| 10000|  Spark|
|   Ram|Sachin|      |  BLR|     F| 10000|English|
|   Ram|Sachin|      |  BLR|     F| 10000|  Maths|
|Aniket|   NaN|Mishra|  HYD|     F|  2000| Civics|
|Aniket|   NaN|Mishra|  HYD|     F|  2000|History|
|Mitali| Sahil|Kashiv|  BLR|     M|  5000| Civics|
|Mitali| Sahil|Kashiv|  BLR|     M|  5000|History|
|  Zaid| Riyaz|Shaikh|  BLR|     M|  4000| Civics|
|  Zaid| Riyaz|Shaikh|  BLR|     M|  4000|History|
|  Sufi|  Alim|Shaikh|  HYD|     M|  1000|  Hindi|
|  Sufi|  Alim|Shaikh|  HYD|     M|  1000|History|
|  Saif|   NaN|Shaikh|  HYD|     M|  1000|English|
|  Saif|   NaN|Shaikh|  HYD|     M|  1000|Science|
|  Saif|   NaN|Shaikh|  HYD|   

In [ ]:
new_df.distinct().show()

+------+------+------+-----+------+------+-------+
| FName| MName| LName|State|Gender|Salary|Subject|
+------+------+------+-----+------+------+-------+
|Aniket|      |Mishra|  HYD|     F|  2000| Civics|
|   Ram|Sachin|      |  BLR|     F| 10000|  Spark|
|  Saif|      |Shaikh|  HYD|     M|  1000|English|
|  Saif|      |Shaikh|  HYD|     M|  1000|  Maths|
|   Ram|Sachin|      |  BLR|     F| 10000|  Maths|
|   Ram|Sachin|      |  BLR|     F| 10000|English|
|Aniket|      |Mishra|  HYD|     F|  2000|History|
|  Saif|      |Shaikh|  HYD|     M|  1000|Science|
|  Zaid| Riyaz|Shaikh|  BLR|     M|  4000| Civics|
|  Zaid| Riyaz|Shaikh|  BLR|     M|  4000|History|
|  Sufi|  Alim|Shaikh|  HYD|     M|  1000|  Hindi|
|Mitali| Sahil|Kashiv|  BLR|     M|  5000| Civics|
|Mitali| Sahil|Kashiv|  BLR|     M|  5000|History|
|  Sufi|  Alim|Shaikh|  HYD|     M|  1000|History|
+------+------+------+-----+------+------+-------+



In [ ]:
a = df.select(explode('Subject').alias('Subject'))
unique = a.dropDuplicates()
unique.show()
print('Unique Counts:', unique.count())

+-------+
|Subject|
+-------+
|Science|
|English|
|History|
| Civics|
|  Maths|
|  Spark|
|  Hindi|
+-------+

Unique Counts: 7


In [ ]:
df.filter(col('Full_Name.'))

# Pandas

In [ ]:
# Required lib and modules
import numpy as np
import pandas as pd

In [ ]:
# Reading data

path = "/content/drive/MyDrive/Colab Notebooks/police_project.csv"
df = pd.read_csv(path)

# Copying the data 
df_copy = df.copy(deep=True)

# displaying the data
df.head()

,stop_date,stop_time,county_name,driver_gender,driver_age_raw,driver_age,driver_race,violation_raw,violation,search_conducted,search_type,stop_outcome,is_arrested,stop_duration,drugs_related_stop
0,2005-01-02,01:55,NaN,M,1985.0,20.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
1,2005-01-18,08:15,NaN,M,1965.0,40.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
2,2005-01-23,23:15,NaN,M,1972.0,33.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False
3,2005-02-20,17:15,NaN,M,1986.0,19.0,White,Call for Service,Other,False,NaN,Arrest Driver,True,16-30 Min,False
4,2005-03-14,10:00,NaN,F,1984.0,21.0,White,Speeding,Speeding,False,NaN,Citation,False,0-15 Min,False


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91741 entries, 0 to 91740
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   stop_date           91741 non-null  object 
 1   stop_time           91741 non-null  object 
 2   county_name         0 non-null      float64
 3   driver_gender       86406 non-null  object 
 4   driver_age_raw      86414 non-null  float64
 5   driver_age          86120 non-null  float64
 6   driver_race         86408 non-null  object 
 7   violation_raw       86408 non-null  object 
 8   violation           86408 non-null  object 
 9   search_conducted    91741 non-null  bool   
 10  search_type         3196 non-null   object 
 11  stop_outcome        86408 non-null  object 
 12  is_arrested         86408 non-null  object 
 13  stop_duration       86408 non-null  object 
 14  drugs_related_stop  91741 non-null  bool   
dtypes: bool(2), float64(3), object(10)
memory usage: 9.3+

In [ ]:
df.describe() #include='all'

,county_name,driver_age_raw,driver_age
count,0.0,86414.000000,86120.000000
mean,NaN,1970.491228,34.011333
std,NaN,110.914909,12.738564
min,NaN,0.000000,15.000000
25%,NaN,1967.000000,23.000000
50%,NaN,1980.000000,31.000000
75%,NaN,1987.000000,43.000000
max,NaN,8801.000000,99.000000


In [ ]:
df.select_dtypes(include='number', exclude='object').head()

,county_name,driver_age_raw,driver_age
0,NaN,1985.0,20.0
1,NaN,1965.0,40.0
2,NaN,1972.0,33.0
3,NaN,1986.0,19.0
4,NaN,1984.0,21.0


In [ ]:
df.columns

Index(['stop_date', 'stop_time', 'county_name', 'driver_gender',
       'driver_age_raw', 'driver_age', 'driver_race', 'violation_raw',
       'violation', 'search_conducted', 'search_type', 'stop_outcome',
       'is_arrested', 'stop_duration', 'drugs_related_stop'],
      dtype='object')

In [ ]:
df.loc[((df.violation=='Speeding') & (df.driver_age>=50)), ['violation', 'driver_age']]\
.sort_values('driver_age')\
.nlargest(5, 'driver_age')\
.reset_index(drop=True)

,violation,driver_age
0,Speeding,90.0
1,Speeding,88.0
2,Speeding,88.0
3,Speeding,87.0
4,Speeding,87.0


In [ ]:
 inbdf['county_name'].count()0nb

0